In [1]:
#pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of the first GPU


True
1
NVIDIA GeForce GTX 1660 SUPER


In [3]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from collections import Counter
from torch.utils.data import WeightedRandomSampler


transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),  # Random rotations
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust colors
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random crop with scaling
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize as per ResNet
])

train_data = datasets.ImageFolder(root='C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\training-images', transform=transform_train)


# Compute class counts dynamically
all_labels = [label for _, label in train_data.samples]  # Extract all labels
class_counts = Counter(all_labels)  # Count occurrences of each class

# Compute class weights
class_weights = 1. / torch.tensor([class_counts[cls] for cls in sorted(class_counts.keys())], dtype=torch.float)

# Compute sample weights
sample_weights = [class_weights[label] for _, label in train_data.samples]

# Define the WeightedRandomSampler
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# Update DataLoader
train_loader = DataLoader(train_data, batch_size=32, sampler=sampler)


train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_data = datasets.ImageFolder(root='C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\validation-images', transform=transform_val)

val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)

# Unfreeze some layers for fine-tuning
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Unfreeze the last few layers
for param in model.layer4.parameters():  # Unfreeze the last residual block
    param.requires_grad = True

model.fc.requires_grad = True  # Unfreeze the fully connected head (already modified)

# Modify the loss function
criterion = nn.BCEWithLogitsLoss()

class CustomClassificationHead(nn.Module):
    def __init__(self, in_features):
        super(CustomClassificationHead, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 1)  # Single output unit (logits)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Replace the ResNet fc layer with the custom classification head
model.fc = CustomClassificationHead(model.fc.in_features)

# Send the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


C:\Users\redfr\elevenvenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\redfr\elevenvenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Define loss function and optimizer
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight('balanced', classes=np.unique(all_labels), y=all_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])  # Adjust for "fake-images"

optimizer = optim.Adam(model.parameters(), lr=1e-6, weight_decay=1e-4)  # Add weight decay
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.1, verbose=True)

# Train the model
num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Reshape labels to match the output shape and ensure they're of type float
        labels = labels.view(-1, 1).float()  # Reshape to (batch_size, 1) and convert to float
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # Print average loss after each epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}")

    # Now evaluate on the validation set
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            labels = labels.view(-1, 1).float()  # Reshape to match model output
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    # Print the validation loss after each epoch
    print(f"Validation Loss: {val_loss/len(val_loader)}")

    # Step the scheduler with the average validation loss
    scheduler.step(val_loss/len(val_loader))  # Pass average validation loss to scheduler


C:\Users\redfr\elevenvenv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/40, Training Loss: 0.6777931577280948
Validation Loss: 0.6524626960915126
Epoch 2/40, Training Loss: 0.6005927282012573
Validation Loss: 0.5482822649934319
Epoch 3/40, Training Loss: 0.4796047139085949
Validation Loss: 0.42730511423577083
Epoch 4/40, Training Loss: 0.3859639907387902
Validation Loss: 0.33171024081412326
Epoch 5/40, Training Loss: 0.3266810616243895
Validation Loss: 0.27653290482049575
Epoch 6/40, Training Loss: 0.2911907239077566
Validation Loss: 0.22599692567345803
Epoch 7/40, Training Loss: 0.26751849716806303
Validation Loss: 0.2189605226844884
Epoch 8/40, Training Loss: 0.242044109655054
Validation Loss: 0.19413894176315727
Epoch 9/40, Training Loss: 0.2158321804394711
Validation Loss: 0.16764079473829002
Epoch 10/40, Training Loss: 0.21332471960946678
Validation Loss: 0.14069254286168667
Epoch 11/40, Training Loss: 0.19377399597410752
Validation Loss: 0.1412876317005479
Epoch 12/40, Training Loss: 0.18234728494961694
Validation Loss: 0.12022504532772503
Ep

In [6]:
# Load the test dataset
test_data = datasets.ImageFolder(root='C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\testing-images', transform=transform_val)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
# Set the model to evaluation mode
model.eval()

# Initialize variables to track accuracy
correct = 0
total = 0

with torch.no_grad(): 
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Get model predictions
        outputs = model(inputs)
        predicted = torch.sigmoid(outputs).round()  # Binary predictions (0 or 1)

        # Ensure labels are in the same format (0 or 1)
        labels = labels.view(-1, 1)  # Reshape labels to (batch_size, 1) if needed

        # Update correct and total counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 68.60%


In [7]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = torch.sigmoid(outputs).round()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Generate confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# Optional: Generate classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=test_data.classes))

Confusion Matrix:
[[231 314]
 [ 28 516]]
Classification Report:
              precision    recall  f1-score   support

 fake-images       0.89      0.42      0.57       545
 real-images       0.62      0.95      0.75       544

    accuracy                           0.69      1089
   macro avg       0.76      0.69      0.66      1089
weighted avg       0.76      0.69      0.66      1089

